### **1. Estruturas Fundamentais da Linguagem (Python Básico)**
*(Notebook : `01_Base_Python_Estruturas`)*

1 - Variáveis** (Guardar valores)

2 - Listas** (Vários valores)

3 - Dicionário** (Estrutura `Chave: Valor`, similar a JSON)

4 - Condição** (`if` para tomada de decisão)

5 - Loop** (`for` para repetição de ações)

6 - Função** (`def` para criar código reutilizável)


###1. Variáveis (guardar valores)

In [0]:
nome = "Maria"
idade = 30
peso = 50

print(nome, idade, peso)

Maria 30 50


###2. Listas (vários valores)

In [0]:
frutas = ["maçã", "banana", "uva"]
print(frutas)

['maçã', 'banana', 'uva']


###3. Dicionário NO PySpark

-  Representar dados estruturados de forma clara usando chave → valor.
-  Facilitar leitura de JSON e APIs.
-  Criar DataFrames onde o nome das colunas já vem do próprio dicionário.
-  Evitar erros de ordem dos campos.
-  Ser mais natural para dados complexos.

####FORMA 1 — Lista de listas + Schema (MELHOR PRÁTICA!)

**Quando usar?**
- Em produção
- Quando você precisa garantir os tipos
- Quando o arquivo real vai seguir esse schema

**Como funciona?**
- Cada lista interna é uma linha
- O schema define o tipo de cada coluna
- O Spark NÃO precisa adivinhar nada

In [0]:

# Exemplo Forma 1

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# DEFININDO O SCHEMA MANUALMENTE (melhor prática)
schema = StructType([
    StructField("nome", StringType()),        # Coluna 1
    StructField("sobrenome", StringType()),   # Coluna 2
    StructField("idade", IntegerType())       # Coluna 3
])

# CADA LISTA = UMA LINHA
dados = [
    ["Maria", "Silva", 25],
    ["João", "Oliveira", 35],
    ["Tereza", "Santos", 29],
    ["Antonio", "Lima", 31]
]

# CRIANDO O DATAFRAME COM SCHEMA DEFINIDO
df = spark.createDataFrame(dados, schema)
df.show()


+-------+---------+-----+
|   nome|sobrenome|idade|
+-------+---------+-----+
|  Maria|    Silva|   25|
|   João| Oliveira|   35|
| Tereza|   Santos|   29|
|Antonio|     Lima|   31|
+-------+---------+-----+



####FORMA 2 — Lista de tuplas + nomes das colunas

**Quando usar?**
- Protótipos
- Estudos
- Quando não precisa de schema detalhado
- Criação muito rápida

**Como funciona?**
- Tupla = linha
- Você só informa o nome das colunas
- O Spark infere automaticamente o tipo

In [0]:

# Exemplo Forma 2       

# LISTA DE TUPLAS
# Tupla = linha + imutável (não deixa você alterar sem querer)
dados = [
    ("Maria", "Silva", 25),
    ("João", "Oliveira", 35),
    ("Tereza", "Santos", 29),
    ("Antonio", "Lima", 31)
]

# NOMES DAS COLUNAS
colunas = ["nome", "sobrenome", "idade"]

# SPARK INFERE O TIPO AUTOMATICAMENTE
df = spark.createDataFrame(dados, colunas)

df.show()


+-------+---------+-----+
|   nome|sobrenome|idade|
+-------+---------+-----+
|  Maria|    Silva|   25|
|   João| Oliveira|   35|
| Tereza|   Santos|   29|
|Antonio|     Lima|   31|
+-------+---------+-----+



####FORMA 3 — Lista de dicionários (parecido com JSON)
**Quando usar?**
- Quando os dados vêm de API (JSON)
- Quando já estão em dicts
- Quando você quer deixar o código legível

**Como funciona?**
- Cada dicionário é uma linha
- As chaves viram colunas
- O Spark infere o tipo

In [0]:
# Exemplo Forma 3

# CADA DICIONÁRIO REPRESENTA UMA LINHA
# A CHAVE SEMPRE VIRA NOME DA COLUNA
dados = [
    {
        "nome": "Maria",
        "sobrenome": "Silva",
        "idade": 25
    },
    {
        "nome": "João",
        "sobrenome": "Oliveira",
        "idade": 35
    },
    {
        "nome": "Tereza",
        "sobrenome": "Santos",
        "idade": 29
    },
    {
        "nome": "Antonio",
        "sobrenome": "Lima",
        "idade": 31
    }
]

# SPARK ENTENDE AS CHAVES COMO COLUNAS E OS VALORES COMO LINHAS
df = spark.createDataFrame(dados)

df.show()


+-----+-------+---------+
|idade|   nome|sobrenome|
+-----+-------+---------+
|   25|  Maria|    Silva|
|   35|   João| Oliveira|
|   29| Tereza|   Santos|
|   31|Antonio|     Lima|
+-----+-------+---------+



## QUAL FORMA USAR? (resumo rápido)

| Forma                | Melhor para             | Controle de tipos | Facilidade |
|----------------------|--------------------------|-------------------|------------|
| 1 — Lista + Schema   | Produção, ingestão séria | ⭐⭐⭐⭐⭐            | ⭐⭐         |
| 2 — Tuplas           | Testes rápidos           | ⭐⭐                | ⭐⭐⭐⭐⭐     |
| 3 — Dicionários      | Dados vindos de JSON     | ⭐⭐⭐               | ⭐⭐⭐⭐      |


###4. Condição (if)

O que é uma condição (if) em Python?
O if é usado quando você quer que o código tome decisões.

Ele funciona como:
- Se uma condição for verdadeira → faz algo
- Senão → faz outra coisa

É a mesma lógica que você usa na vida real:
- Se chover, levo guarda-chuva.
- Senão, saio sem.

Usar lógica condicional o tempo todo, por exemplo:
- Validar dados
- Tratar valores nulos
- Criar colunas derivadas
- Fazer regras para qualidade de dados
- Testar pipelines

**Tópicos**
1. elif (mais de uma condição)
2. operadores lógicos (and / or)
3. if dentro de função
4. como usar if no PySpark (when, otherwise)

In [0]:
idade = 17

if idade >= 18:
    print("Maior de idade")
else:
    print("Menor de idade")


Menor de idade


In [0]:
### if aplicada dentro de um DataFrame.

from pyspark.sql.functions import when, col

df = df.withColumn(
    "classificacao",
    when(col("idade") >= 18, "adulto").otherwise("menor")
)


In [0]:
## 1. elif — Quando você tem mais de uma condição

## O elif significa “senão, se…”.
## Ele é usado quando você precisa testar mais de duas possibilidades.

#Exemplo 
idade = 17

if idade < 12:
    print("Criança")
elif idade < 18:
    print("Adolescente")
elif idade < 60:
    print("Adulto")
else:
    print("Idoso")

## Obs:
## O Python testa de cima pra baixo.
## Assim que encontra uma condição verdadeira, ele para e executa só aquele bloco.
## O elif evita que você escreva vários if separados (o que seria errado).

Adolescente


In [0]:
## 2. Operadores lógicos (and / or)
## and = as duas condições precisam ser verdadeiras

## Exemplo: pessoa deve ser maior de 18 E ter CNH
idade = 20
tem_cnh = True

if idade >= 18 and tem_cnh:
    print("Pode dirigir")
else:
    print("Não pode dirigir")

## or = basta UMA condição ser verdadeira
## Exemplo: pode ter desconto se for estudante OU idoso

estudante = False
idoso = True

if estudante or idoso:
    print("Tem direito a desconto")
else:
    print("Não tem desconto")

Pode dirigir
Tem direito a desconto


In [0]:
## 3. If dentro de função
## Você usa if dentro de funções quando quer que elas tomem decisões.

## Exemplo 
def calcular_preco(valor, eh_vip):
    if eh_vip:
        return valor * 0.8  # 20% de desconto
    else:
        return valor

preco = calcular_preco(100, True)
print(preco)  # 80

## Obs:
## - A função recebe valores.
## - O if decide o que fazer com eles.
## - E retorna o resultado.

80.0


In [0]:
## 4. If no PySpark (when, otherwise)
## Em PySpark você não usa if direto, porque Spark trabalha com colunas, não com valores individuais.

## when → equivalente ao if
## otherwise → equivalente ao else

## Exemplo  (classificar idade)
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# criar/get SparkSession (em Databricks já existe 'spark', então essa linha pode ser desnecessária)
spark = SparkSession.builder.getOrCreate()

# criar DataFrame de exemplo com uma coluna 'idade'
df = spark.createDataFrame(
    [(10,), (15,), (30,), (80,)],
    ["idade"]
)
# agora aplica a lógica de faixa etária
df = df.withColumn(
    "faixa_etaria",
    F.when(F.col("idade") < 12, "Criança")
     .when(F.col("idade") < 18, "Adolescente")
     .when(F.col("idade") < 60, "Adulto")
     .otherwise("Idoso")
)
df.show()

## Obs:
## É exatamente igual ao:
## - if
## - elif
## - elif
## - else



+-----+------------+
|idade|faixa_etaria|
+-----+------------+
|   10|     Criança|
|   15| Adolescente|
|   30|      Adulto|
|   80|       Idoso|
+-----+------------+



###5. Loop (for)

O loop for serve para repetir uma ação várias vezes.
Ele “passa por cada item” de uma lista, tupla, string, etc.

Você usa para:
- percorrer uma lista
- executar algo várias vezes
- gerar valores repetidos
- criar dados para teste
- montar listas para DataFrames

Como o “for” aparece no PySpark na prática:
1. Quando criamos uma lista para um DataFrame:
2. Quando geramos colunas repetitivas para teste:
3. Quando lemos vários arquivos:

In [0]:
## 1. Quando criamos uma lista para um DataFrame:
## Usamos for para gerar dados automaticamente, em vez de digitar um por um.

import pandas as pd

# 1. Criar lista de dados automaticamente
dados = []
for i in range(5):
    dados.append((i, i*10))

# 2. Transformar a lista em DataFrame
df = pd.DataFrame(dados, columns=["id", "valor"])
print(df)

## Obs: Serve para criar dados de teste.

   id  valor
0   0      0
1   1     10
2   2     20
3   3     30
4   4     40


In [0]:
## 2. Quando geramos colunas repetitivas para teste:

"""
Explicação: Em vez de escrever
    
df = df.select(
    F.lit(0).alias("col_0"),
    F.lit(1).alias("col_1"),
    F.lit(2).alias("col_2"),
    ...
    F.lit(50).alias("col_50")
)
Você usa um for que cria tudo automaticamente.

Está fazendo exatamente isso:
- Criando 5 colunas automaticamente
- Cada coluna tem um valor fixo (lit(i))
- Cada coluna tem nome automático (col_0, col_1, ...)

Depois o select monta o DataFrame com essas colunas(como no exemplo abaixo)

colunas = []
for i in range(5):
    colunas.append(F.lit(i).alias(f"col_{i}"))

"""

from pyspark.sql import functions as F

# 1. Criando lista de colunas automáticas
colunas = []
for i in range(5):
    colunas.append(F.lit(i).alias(f"col_{i}"))

# 2. Criando um DF simples com 1 linha
df = spark.createDataFrame([(1,)])

# 3. Substituindo pela lista de colunas
df = df.select(*colunas)

df.show()


+-----+-----+-----+-----+-----+
|col_0|col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+-----+
|    0|    1|    2|    3|    4|
+-----+-----+-----+-----+-----+



In [0]:
## 3. Quando lemos vários arquivos:
## Você tem vários arquivos para ler (ex: jan.csv, fev.csv, mar.csv).
arquivos = ["jan.csv", "fev.csv", "mar.csv"]


"""
- Sem for:
Você teria que fazer:
    spark.read.csv("jan.csv")
    spark.read.csv("fev.csv")
    spark.read.csv("mar.csv")

- Com for:
Você deixa o Python fazer isso sozinho:
arquivos = ["jan.csv", "fev.csv", "mar.csv"]

for arquivo in arquivos:
    df = spark.read.csv(arquivo)

    

"""


# Lista com os nomes dos arquivos que queremos ler
arquivos = ["dados_1.csv", "dados_2.csv", "dados_3.csv"]

# Lista vazia onde vamos guardar cada DataFrame lido
dfs = []

# Loop para ler cada arquivo da lista
for arquivo in arquivos:
    
    # Lê um arquivo CSV e cria um DataFrame temporário
    df_temp = spark.read.csv(arquivo, header=True)
    
    # Coloca o DataFrame temporário dentro da lista
    dfs.append(df_temp)

# Começamos o DataFrame final com o primeiro DataFrame da lista
df_final = dfs[0]

# Fazemos UNION com os demais DataFrames (juntando tudo)
for df in dfs[1:]:
    df_final = df_final.unionByName(df)

# Mostra o DataFrame final já com todos os arquivos juntos
# df_final.show()


##O for lê todos os arquivos automaticamente, e depois fazemos um “union” para juntar tudo.



###6. Função

Uma função é um bloco de código que você cria para reutilizar em vários lugares.

Em vez de repetir o mesmo código 10 vezes, você cria UMA função e chama quando precisar.Uma função é um bloco de código que você cria para reutilizar em vários lugares.

Em vez de repetir o mesmo código 10 vezes, você cria UMA função e chama quando precisar.



In [0]:
# Exemplo 1 — Função simples

# Criamos uma função chamada processar_df
def processar_df(df):
    # Remove linhas duplicadas
    df = df.dropDuplicates()
    # Remove linhas com valores nulos
    df = df.na.drop()
    # Retorna o DataFrame limpo
    return df

In [0]:
## Exemplo 2. Como usar a função 

# Aqui estamos chamando a função processar_df
# Passando como argumento o DataFrame df_original

#df_limpo = processar_df(df_original)

# Agora df_limpo recebe o resultado da função (um DataFrame limpo)

In [0]:
# Exemplo 3: função mais completa

# Criamos uma função chamada limpar_e_renomear
def limpar_e_renomear(df):
    # Remove linhas duplicadas
    df = df.dropDuplicates()
    # Remove linhas com valores nulos
    df = df.na.drop()
    # Renomeia a coluna "col1" para "id"
    df = df.withColumnRenamed("col1", "id")
    # Retorna o DataFrame modificado
    return df


#### Exercícios

In [0]:
"""
EXERCÍCIO 1 — Função simples em Python
Crie uma função chamada dobrar que recebe um número e retorna ele vezes 2.
use: 

def nome_da_funcao(parametro):
    return alguma_coisa

"""

# Resultado:
# Função que dobra um número
# Recebe um número e devolve ele multiplicado por 2.

# Codigo limpo
# def dobrar(numero):
#     return numero * 2

# Perfumaria nao roda 
# 'def' cria a função, 'dobrar' é o nome, e 'numero' é o valor que ela recebe.
def dobrar(numero):
    # A função calcula o valor recebido (numero) vezes 2, e devolve o resultado.
    return numero * 2

# CHAMANDO A FUNÇÃO
# A função 'dobrar' é chamada com o valor 5.
# O resultado (10) é guardado na variável 'resultado'.
resultado = dobrar(5)
# Exibe o valor que está em 'resultado'.
print(resultado)

10


In [0]:
"""
EXERCÍCIO 2 — Função para somar 2 números
Crie uma função:
somar(a, b)
que retorna a + b.
Dica:
A função deve ter 2 parâmetros.

"""

# Resultado
# Função que soma dois números
# Recebe dois valores e devolve a soma.

# Codigo limpo
# def somar(a, b):
#     return a + b



# Perfumaria nao roda 
# 'area_quadrado' é o nome. Ela recebe apenas um valor: o 'lado'.
def area_quadrado(lado):
    # O cálculo é lado * lado. O resultado é devolvido.
    return lado * lado

# A função é chamada com 4 (que é o lado).
# O resultado (4 * 4 = 16) é armazenado.
resultado = area_quadrado(4)

# Mostra o valor 16 (a área do quadrado de lado 4).
print(f"A área do quadrado é: {resultado}")


A área do quadrado é: 16


In [0]:
"""
EXERCÍCIO 3 — Função que limpa um DataFrame no PySpark
Crie uma função chamada: limpar(df)
Ela deve:
- remover duplicados
- remover nulos
- retornar o DataFrame limpo
Dica:
Use:
df.dropDuplicates()
df.na.drop()

"""

# Resultado
# Função que limpa um DataFrame no PySpark
# Essa função é muito comum em ETL.

# Codigo limpo 
def limpar(df):
    df = df.dropDuplicates()   # Remove linhas duplicadas
    df = df.na.drop()          # Remove linhas com valores nulos
    return df                  # Retorna o DataFrame limpo

# Perfumaria nao roda
# Dados de entrada: uma lista de registros (id, nome).
# dados = [
#     (1, "Maria"),
#     (1, "Maria"),  # Exemplo de dado REPETIDO
#     (2, None)      # Exemplo de dado VAZIO (nulo)
# ]
# # Nomes dados às colunas da tabela.
# colunas = ["id", "nome"]

# # CRIAÇÃO DA TABELA (DATAFRAME)
# # Pega os 'dados' e 'colunas' e cria a tabela 'df' no PySpark.
# df = spark.createDataFrame(dados, colunas)
# # Mostra a tabela 'df' ORIGINAL (com lixo).
# df.show()

# # LIMPEZA
# # 'limpar' é uma função que remove o lixo (duplicatas e nulos) de 'df'.
# # O resultado da limpeza é salvo em uma NOVA tabela chamada 'df_limpo'.
# df_limpo = limpar(df)
# # Mostra a tabela 'df_limpo' DEPOIS de ter sido limpa.
# df_limpo.show()


In [0]:
"""
EXERCÍCIO 4 — Função que cria coluna com faixa etária
Crie uma função chamada: adicionar_faixa(df)

Que adiciona uma coluna chamada faixa_etaria com:
- idade < 12 → "Criança"
- idade < 18 → "Adolescente"
- idade < 60 → "Adulto"
- senão → "Idoso"

🔎 Dica:
Use:
F.when().otherwise()

"""

# Resultado
# Função que adiciona coluna de faixa etária
# É o equivalente ao if/elif/else no PySpark.


# Codigo limpo
# from pyspark.sql import functions as F

# def adicionar_faixa(df):
#     return df.withColumn(
#         "faixa_etaria",
#         F.when(F.col("idade") < 12, "Criança")
#          .when(F.col("idade") < 18, "Adolescente")
#          .when(F.col("idade") < 60, "Adulto")
#          .otherwise("Idoso")
#     )

# Perfumaria 
# Importa as funções do PySpark e dá a elas o apelido 'F'.
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# CRIAÇÃO DOS DADOS DE TESTE (df_entrada) 
# Dados de exemplo: (id, idade)
dados_teste = [
    (1, 10),  # Criança
    (2, 17),  # Adolescente
    (3, 35),  # Adulto
    (4, 65)   # Idoso
]
colunas = ["id", "idade"]

# Cria o DataFrame (Tabela) de entrada que será passado para a função
df_entrada = spark.createDataFrame(dados_teste, colunas)

# DEFINIÇÃO DA SUA FUNÇÃO ---
def adicionar_faixa(df):
    return df.withColumn(
        "faixa_etaria",
        F.when(F.col("idade") < 12, "Criança")
        .when(F.col("idade") < 18, "Adolescente")
        .when(F.col("idade") < 60, "Adulto")
        .otherwise("Idoso")
    )

# EXECUÇÃO E IMPRESSÃO ---
# Chama a função, passando o DataFrame de entrada e salva o resultado
df_resultado = adicionar_faixa(df_entrada)

# Imprime o DataFrame resultante na tela
print("--- Resultado Final da Classificação ---")
df_resultado.show()

--- Resultado Final da Classificação ---
+---+-----+------------+
| id|idade|faixa_etaria|
+---+-----+------------+
|  1|   10|     Criança|
|  2|   17| Adolescente|
|  3|   35|      Adulto|
|  4|   65|       Idoso|
+---+-----+------------+



In [0]:
"""
EXERCÍCIO 5 — Função que lê vários arquivos
Crie uma função: ler_varios_arquivos(caminhos)

Que recebe uma lista de caminhos e retorna um único DataFrame com:
df = spark.read.csv(caminho)

e junta tudo com:
df_final = df_final.union(df)

"""

# Resultado
# Função que lê vários arquivos e junta tudo
# Lê vários arquivos e faz um “union” de todos.


#codigo limpo
def ler_varios_arquivos(caminhos):
    df_final = None   # Começa vazio

    for caminho in caminhos:
        df_temp = spark.read.csv(caminho, header=True)

        if df_final is None:
            df_final = df_temp            # Primeiro arquivo
        else:
            df_final = df_final.union(df_temp)   # Junta com os outros

    return df_final



#perfumaria nao roda 
# from pyspark.sql import functions as F

# # 1. DADOS DE TESTE (Criados diretamente no ambiente Spark do Databricks)
# # Criamos duas tabelas (DataFrames) que sua função irá unir.
# # NOTE: Usamos o objeto 'spark' que já está ativo no seu cluster.
# df_arq1 = spark.createDataFrame([(1, "A")], ["id", "dado"])
# df_arq2 = spark.createDataFrame([(2, "B")], ["id", "dado"])

# # Lista das tabelas que a função irá percorrer e unir.
# tabelas_para_unir = [df_arq1, df_arq2]

# # 2. SUA FUNÇÃO (LÓGICA DE UNION)
# def ler_varios_arquivos(tabelas):
#     df_final = None   
    
#     # Percorre cada tabela na lista
#     for df_temp in tabelas:
        
#         # Se for a primeira tabela, inicia o DataFrame final
#         if df_final is None:
#             df_final = df_temp   
        
#         # Senão, empilha as linhas (UNION)
#         else:
#             df_final = df_final.union(df_temp)

#     return df_final

# # 3. EXECUÇÃO E IMPRESSÃO
# df_resultado = ler_varios_arquivos(tabelas_para_unir)

# print("--- Resultado Final da União ---")
# df_resultado.show()